In [6]:
import glob, json

with open(f"./data/opendialkg/entity2id.json", 'r', encoding="utf-8") as f:
    entity2id = json.load(f)

def get_user_preference(sample_result_path):
    sample_result = json.load(open(sample_result_path))
    dialog = sample_result['simulator_dialog']['context']
    rec_label = sample_result['rec']
    orig_user_uttrs = ''
    extened_user_uttrs = ''
    for uttr in dialog:
        if 'rec_success' in uttr: 
            break
        if uttr['role'] == 'user':
            user_uttr = uttr['content']
            orig_user_uttrs += user_uttr
    
    for uttr in dialog:
        extened_user_uttrs += uttr['content']
        
    return orig_user_uttrs, extened_user_uttrs
        
folder_path = "/home/work/shchoi/iEvaLM-CRS/save_5/user_Llama-3.1-8B-Instruct/emb_Snowflake/snowflake-arctic-embed-s/openmodel_Llama-3.2-1B-Instruct_top10_full_history/opendialkg_eval/full_non_repeated/test"
results_path = glob.glob(folder_path + '/*.json')
print(f'The number of results: {len(results_path)}')

orig_user_uttrs_list = []
extened_user_uttrs_list = []
for result_path in results_path:
    orig_user_uttrs, extended_user_uttrs = get_user_preference(result_path)
    orig_user_uttrs_list.append(orig_user_uttrs)
    extened_user_uttrs_list.append(extended_user_uttrs)

The number of results: 789


In [8]:
import nltk, statistics
from nltk import ngrams
from collections import Counter

def calculate_ngram_specificity(text, n):
    # Tokenize the text
    tokens = nltk.word_tokenize(text.lower())
    
    # Generate n-grams
    n_grams = list(ngrams(tokens, n))
    
    # Count the occurrences of each n-gram
    ngram_counts = Counter(n_grams)
    
    # Calculate the total number of n-grams
    total_ngrams = len(n_grams)
    
    # Calculate specificity for each n-gram
    specificity = {gram: count / total_ngrams for gram, count in ngram_counts.items()}
    
    # how to calculate the the number of unique n-grams
    num_unique_ngrams = len(ngram_counts)
    
    return num_unique_ngrams

for n in range(2, 5):
    orig_Ngram_specificity = []
    extened_Ngram_specificity = []
    for orig_user_uttrs, extended_user_uttrs in zip(orig_user_uttrs_list, extened_user_uttrs_list):
        orig_Ngram_specificity.append(calculate_ngram_specificity(orig_user_uttrs, n))
        extened_Ngram_specificity.append(calculate_ngram_specificity(extended_user_uttrs, n))
    print(f'Ngram: {n}')
    print(f'Original: mean:{statistics.mean(orig_Ngram_specificity)}; std:{statistics.stdev(orig_Ngram_specificity)}')
    print(f'Extended: mean:{statistics.mean(extened_Ngram_specificity)}; std:{statistics.stdev(extened_Ngram_specificity)}')    
    # print(statistics.mean)
    

Ngram: 2
Original: mean:19.309252217997464; std:15.127065133958272
Extended: mean:369.3092522179975; std:104.78668531528216
Ngram: 3
Original: mean:18.65272496831432; std:15.800984943329885
Extended: mean:440.4942965779468; std:135.55337432083616
Ngram: 4
Original: mean:17.7617237008872; std:15.998937308291069
Extended: mean:477.8998732572877; std:151.8741425995197
